# Training Regression - Reaction

# Import packages

In [1014]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

/mnt/d/workspace/BACKUP-COPY-/chemprop_2/examples
/mnt/d/workspace/BACKUP-COPY-/chemprop_2


In [1015]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

In [1016]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

## Load data

In [1017]:
train_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_normal.npz', allow_pickle=True)
train_v_normal = train_npz_normal['node_attrs']
train_e_normal = train_npz_normal['edge_attrs']
train_idx_g_normal = train_npz_normal['edge_indices']
train_y_normal = train_npz_normal['ys'] 

val_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_normal.npz', allow_pickle=True)
val_v_normal = val_npz_normal['node_attrs']
val_e_normal = val_npz_normal['edge_attrs']
val_idx_g_normal = val_npz_normal['edge_indices']
val_y_normal = val_npz_normal['ys'] 

test_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_normal.npz', allow_pickle=True)
test_v_normal = test_npz_normal['node_attrs']
test_e_normal = test_npz_normal['edge_attrs']
test_idx_g_normal = test_npz_normal['edge_indices']
test_y_normal = test_npz_normal['ys'] 

In [1018]:
train_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_rc.npz', allow_pickle=True)
train_v_rc = train_npz_rc['node_attrs']
train_e_rc = train_npz_rc['edge_attrs']
train_idx_g_rc = train_npz_rc['edge_indices']
train_y_rc = train_npz_rc['ys'] 

val_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_rc.npz', allow_pickle=True)
val_v_rc = val_npz_rc['node_attrs']
val_e_rc = val_npz_rc['edge_attrs']
val_idx_g_rc = val_npz_rc['edge_indices']
val_y_rc = val_npz_rc['ys'] 

test_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_rc.npz', allow_pickle=True)
test_v_rc = test_npz_rc['node_attrs']
test_e_rc = test_npz_rc['edge_attrs']
test_idx_g_rc = test_npz_rc['edge_indices']
test_y_rc = test_npz_rc['ys'] 

In [1019]:
train_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_super.npz', allow_pickle=True)
train_v_super = train_npz_super['node_attrs']
train_e_super = train_npz_super['edge_attrs']
train_idx_g_super = train_npz_super['edge_indices']
train_y_super = train_npz_super['ys'] 

val_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_super.npz', allow_pickle=True)
val_v_super = val_npz_super['node_attrs']
val_e_super = val_npz_super['edge_attrs']
val_idx_g_super = val_npz_super['edge_indices']
val_y_super = val_npz_super['ys'] 

test_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_super.npz', allow_pickle=True)
test_v_super = test_npz_super['node_attrs']
test_e_super = test_npz_super['edge_attrs']
test_idx_g_super = test_npz_super['edge_indices']
test_y_super = test_npz_super['ys'] 

In [1020]:
train_v = np.concatenate((train_v_normal, train_v_rc, train_v_super), axis=0)
train_e = np.concatenate((train_e_normal, train_e_rc, train_e_super), axis=0)
train_idx_g = np.concatenate((train_idx_g_normal, train_idx_g_rc, train_idx_g_super), axis=0)
train_y = np.concatenate((train_y_normal, train_y_rc, train_y_super), axis=0)

In [1021]:
val_v = np.concatenate((val_v_normal, val_v_rc, val_v_super), axis=0)
val_e = np.concatenate((val_e_normal, val_e_rc, val_e_super), axis=0)
val_idx_g = np.concatenate((val_idx_g_normal, val_idx_g_rc, val_idx_g_super), axis=0)
val_y = np.concatenate((val_y_normal, val_y_rc, val_y_super), axis=0)

In [1022]:
test_v = np.concatenate((test_v_normal, test_v_rc, test_v_super), axis=0)
test_e = np.concatenate((test_e_normal, test_e_rc, test_e_super), axis=0)
test_idx_g = np.concatenate((test_idx_g_normal, test_idx_g_rc, test_idx_g_super), axis=0)
test_y = np.concatenate((test_y_normal, test_y_rc, test_y_super), axis=0)

In [1023]:
print(train_idx_g.shape, val_y.shape, test_y.shape)

(12648,) (1581,) (1578,)


## Perform data splitting for training, validation, and testing

In [1024]:
train_dset = data.ReactionDataset(train_v, train_e, train_idx_g, train_y)
print(train_dset[0][3])
scaler = train_dset.normalize_targets()
# print(scaler)
print(train_dset[0][3])

val_dset = data.ReactionDataset(val_v, val_e, val_idx_g, val_y)
val_dset.normalize_targets(scaler)
test_dset = data.ReactionDataset(test_v, test_e, test_idx_g, test_y)

[21.74475479]
[[21.74475479]
 [19.75370979]
 [17.10799026]
 ...
 [18.88179398]
 [13.98910904]
 [29.49899864]]
[[ 0.07020871]
 [-0.1344098 ]
 [-0.40630883]
 ...
 [-0.22401607]
 [-0.7268344 ]
 [ 0.86710775]]
[0.07020871]
[[-1.18898438]
 [-0.89972892]
 [-0.06001916]
 ...
 [-1.47195955]
 [ 2.61759346]
 [ 0.32106197]]


## Get ReactionDatasets

In [1025]:
edge_index=train_dset[1][0][-2]
print(f'edge_index: {edge_index}')
reverse_index=train_dset[1][0][-1]
print(f'reverse_index: {reverse_index}')

edge_index: [[ 0  1  1  2  1  3  3  4  3  8  3 20  4  5  5  6  5 19  6  7  8  9  8 13
   9 10 10 11 11 12 12 13 14 15 15 16 16 17 16 19 16 26 17 18 19 20 20 21
  21 22 21 23 23 24 24 25 25 26]
 [ 1  0  2  1  3  1  4  3  8  3 20  3  5  4  6  5 19  5  7  6  9  8 13  8
  10  9 11 10 12 11 13 12 15 14 16 15 17 16 19 16 26 16 18 17 20 19 21 20
  22 21 23 21 24 23 25 24 26 25]]
reverse_index: [ 1  0  3  2  5  4  7  6  9  8 11 10 13 12 15 14 17 16 19 18 21 20 23 22
 25 24 27 26 29 28 31 30 33 32 35 34 37 36 39 38 41 40 43 42 45 44 47 46
 49 48 51 50 53 52 55 54 57 56]


In [1026]:
import numpy as np

np.arange(6).reshape(-1,2)[:, ::-1].ravel()

array([1, 0, 3, 2, 5, 4])

## Get dataloaders

In [1027]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [1028]:
train_v[0].shape[1]

23

In [1029]:
fdims = (train_v[0].shape[1],train_e[0].shape[1]) # the dimensions of the featurizer, given as (atom_dims, bond_dims).
mp = nn.BondMessagePassing(*fdims)

In [1030]:
print(*fdims)

23 29


## Aggregation

In [1031]:
print(nn.agg.AggregationRegistry)

ClassRegistry {
    'mean': <class 'chemprop.nn.agg.MeanAggregation'>,
    'sum': <class 'chemprop.nn.agg.SumAggregation'>,
    'norm': <class 'chemprop.nn.agg.NormAggregation'>
}


In [1032]:
agg = nn.MeanAggregation()

## Feed-Forward Network (FFN)

In [1033]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [1034]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [1035]:
ffn = nn.RegressionFFN(output_transform=output_transform)

## Batch norm

In [1036]:
batch_norm = True

## Metrics

In [1037]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [1038]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] 
# Only the first metric is used for training and early stopping

## Construct MPNN

In [1039]:
use_fa_layer=True

In [1040]:
# mpnn = models.MPNN_1(mp, agg, ffn, batch_norm, metric_list, use_fa_layer)
# mpnn

# Training and testing

## Set up trainer

In [1041]:
# trainer = pl.Trainer(
#     logger=False,
#     enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
#     enable_progress_bar=True,
#     accelerator="auto",
#     devices=1,
#     max_epochs=100,  # number of epochs to train for
# )

## Start training

In [1042]:
# trainer.fit(mpnn, train_loader, val_loader)

## Test results

In [1043]:
# results = trainer.test(mpnn, test_loader)

In [1044]:
# results

In [1045]:
k_jump=3,


In [1046]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)


In [1047]:
# mpnn = models.MPNN_Modified(
#     message_passing=mp,
#     predictor=ffn,        
#     k_jump=k_jump,
#     metrics=metric_list
#     # init_lr=1e-4,
#     # max_lr=1e-3,
# )

# print("Khởi tạo mô hình thành công!")

In [1048]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [1049]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=10,  # number of epochs to train for
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [1050]:
# trainer = pl.Trainer(
#     max_epochs=10,
#     accelerator="auto", 
#     devices = 1,
#     logger=False, # Bật logging
#     enable_progress_bar=True,
# )

trainer.fit(mpnn, train_loader, val_loader)
results = trainer.test(mpnn, test_loader)

/root/anaconda3/envs/chem_new/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /mnt/d/workspace/BACKUP-COPY-/chemprop_2/examples/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# trainer = pl.Trainer(max_epochs=10, accelerator="auto", logger=True, enable_progress_bar=True)


In [ ]:
# # ===================================================================
# # THAY THẾ TOÀN BỘ PHẦN "Training and testing" BẰNG ĐOẠN MÃ NÀY
# # ===================================================================

# # --- Thiết lập các tham số cho Ensemble ---
# ENSEMBLE_SIZE = 5
# all_test_results = []
# output_dir = Path("./reaction_ensemble_results")
# output_dir.mkdir(exist_ok=True)

# print(f"Bắt đầu huấn luyện ensemble với kích thước = {ENSEMBLE_SIZE}")
# print("-" * 30)

# # --- Bắt đầu vòng lặp huấn luyện Ensemble ---
# for i in range(ENSEMBLE_SIZE):
#     print(f"\n--- Đang huấn luyện mô hình {i+1}/{ENSEMBLE_SIZE} ---")
    
#     # 1. Tạo một mô hình MPNN mới cho mỗi lần lặp để đảm bảo trọng số được khởi tạo lại
#     mpnn = models.MPNNWithSkip(mp, agg, ffn, batch_norm, metric_list)

#     # 2. Tạo một Trainer mới, chỉ định nơi lưu checkpoint cho từng mô hình
#     model_checkpoint_dir = output_dir / f"model_{i}"
#     checkpoint_callback = pl.callbacks.ModelCheckpoint(
#         dirpath=model_checkpoint_dir,
#         monitor="val_loss",
#         mode="min",
#         save_top_k=1,
#         filename='best_model'
#     )
    
#     trainer = pl.Trainer(
#         logger=False,
#         enable_checkpointing=True,
#         callbacks=[checkpoint_callback],
#         enable_progress_bar=True,
#         accelerator="auto",
#         devices=1,
#         max_epochs=200,
#     )
    
#     # 3. Huấn luyện mô hình
#     trainer.fit(mpnn, train_loader, val_loader)
    
#     # 4. Chạy kiểm tra (test) trên mô hình tốt nhất vừa được lưu
#     # và lưu kết quả của lần lặp này
#     print(f"--- Đang kiểm tra mô hình {i+1} ---")
#     best_model_path = checkpoint_callback.best_model_path
#     results = trainer.test(mpnn, test_loader, ckpt_path=best_model_path)
#     all_test_results.append(results[0]) # results là một list, lấy phần tử đầu tiên

# # --- Tổng hợp kết quả ---
# print("\n" + "="*30)
# print("HUẤN LUYỆN ENSEMBLE HOÀN TẤT!")
# print("="*30)

# # Chuyển danh sách kết quả thành DataFrame để dễ tính toán
# results_df = pd.DataFrame(all_test_results)

# print("\n--- Kết quả kiểm tra của từng mô hình ---")
# print(results_df)

# print("\n--- Kết quả trung bình của Ensemble ---")
# print(results_df.mean())